# Linear Paul Trap Tracking algorithm

Code useful to treat videos from Linear Paul trap experiments.

Apart from things that come with Anaconda, one might need to install trackpy and its dependency, pims (to handle videos):

- conda update conda
- conda install -c conda-forge trackpy
- conda install -c conda-forge pims

#### Based on Trackpy Library,  more information on:

http://soft-matter.github.io/trackpy/dev/installation.html


## Imports

In [ ]:
# -*- coding: utf-8 -*-

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from scipy import signal

import pims
import trackpy as tp

import multiprocessing as mpr

## Functions

In [ ]:
def customlocate(arg):
    '''
    Function to replace multiprocessing done by tp.batch function 
    in case it throws any errors. 
    '''
    faux = tp.locate(arg, psize, minmass=minmass, invert=inv)
    shared_f.append(faux)

@pims.pipeline
def as_gray(frame):
    '''
    Function to define how PIMS library deals with color videos to get
    grayscale images needed for the algorithm to work.
    '''
    red = frame[:, :, 0]
    green = frame[:, :, 1]
    blue = frame[:, :, 2]
    return 0.3 * red + 0.4 * green + 0.3 * blue
    #return green
    
@pims.pipeline
def crop(img, a, b, c, d):
    """
    Crop the image to select the region of interest
    """
    x_min = a
    x_max = b
    y_min = c
    y_max = d
    return img[y_min:y_max,x_min:x_max]    

## Inputs

In [ ]:
# Full path to source. Must end in name.extension if video or in *.ext is img sequence
vsrc = '/home/raulrica/Documentos/TESTS/Linear_Paul_Trap/17032023/freq_dist_1Hzdrive.avi'

# Swap this two lines if already loading a grayscale image 
frames = as_gray(pims.open(vsrc))
#frames = pims.open(vsrc)          

%matplotlib notebook
plt.figure()
plt.imshow(50*frames[0], cmap='gray', vmin=0,vmax=255)

In [ ]:
# Region of Interest Selection: y axis goes from yt to yb; x axis go from xl to xr 
yt = 130
yb = 250
xl = 500
xr = 1500

frames = crop(frames, xl, xr, yt, yb)

apsize =  3                                 #approximated measured size of particle in pixels
psize  =  int(1.15*apsize)+1                #correction not to fall short
psize  =  psize if psize%2!=0 else psize+1  #making it odd for the algorithm

#Particle filtering
inv   = False    # colors inversion - False = look for bright spots in dark background
xch   = False    # extra characterization: eccentricity, signal...

## Test Detection

In [ ]:
# Locate characteristics in the test frame:
f = tp.locate(frames[0], 
              psize, 
              invert= inv, 
              characterize = xch)

#Plot to check if some further tuning is needed
plt.figure()
plt.axis('off')
plt.imshow(frames[0], cmap='gray')
plt.plot(f['x'], f['y'],
         'o',
         markersize = psize,
         markerfacecolor='none',
          )

## Brightness over time

In [ ]:
f2check = np.random.choice(np.arange(0,len(frames)),  # within all frames
                           np.min((20,len(frames))),  # take 20 (at most) randomly,
                           replace=True)              # non repeated,
f2check = np.sort(f2check)                            # and ordered 


#Check how brightness of the most (and less) bright particle detected change in time:     
first = []
last  = []
for i in f2check:
    print("checking frame no ", i)
    f = tp.locate(frames[i], psize, invert= inv, 
                                      characterize = xch)
    first.append(np.max(f['mass']))
    try:
        last.append(np.min(f['mass']))
    except:
        pass
plt.figure()
plt.plot(first)
plt.plot(last)    
plt.show()

## Batch Detection

In [ ]:
minmass = 20

maxdis =  10  # max displacement estimated for the particles to track
mem    =  50  # number of frames a particle can be gone for before reapearing

sf     =  0   # start frame
ef     =  100 # end frame, -1 for full video

try:
    f = tp.batch(frames[sf:ef], psize, 
                                invert= inv,
                                characterize = xch,
                                minmass = minmass,
                                processes = 'auto')
except Exception as e:
    print(e)
    print('\nTrying custom pool now...')
    try: 
        manager = mpr.Manager()
        shared_f = manager.list()
        pool = mpr.Pool()
        pool.imap(customlocate, frames[sf:ef], chunksize = 10)
        pool.close()
        pool.join()
        f = pd.concat(shared_f, sort=False) 
        
    except Exception as e:
        print(e)
        print('\nEverything failed. Trying Single Core Processing...') 
        f = tp.batch(frames[sf:ef], psize, 
                                    invert = inv,
                                    characterize = xch,
                                    minmass = minmass,
                                    processes = 1)
  
t = tp.link(f, maxdis, memory=mem)
plt.figure()
tp.plot_traj(t,label = True)
#plt.gca().set_aspect('equal', 'box')

## Trajectory Output

In [ ]:
# Save the columns 'frame', 'x' and 'y' to a txt file in the same folder than the input video/img sequence.
# If eccentricity, e.g is stored too, we can add it to the output by just expanding the list below to
# t[['frame','x','y','ecc']].to_cs.....

t[['frame','x','y']].to_csv(vsrc[:-4]+'_Trajectory.txt', 
                            sep='\t', 
                            index=False, 
                            header = ['#columns: frame','x','y'])#False)

## Further data treatment example:

In [ ]:
# Retrieve x and y traces of particle npart from the dataframe t and plotting the traces and position histograms 
fps=25
bins = 30

npart = 4
taux  = t.loc[t['particle']==npart]

time = taux['frame'].to_numpy()/fps
x    = taux['x'].to_numpy()
y    = taux['y'].to_numpy()

fig, axs = plt.subplots(2, 2,num='Info')
fig.tight_layout(pad=2)

axs[0,0].set_xlabel('Posiciones x')
axs[0,1].set_xlabel('Posiciones y')
axs[0,0].plot(time,x)
axs[0,1].plot(time,y)
axs[1,0].hist(x, bins=bins)
axs[1,1].hist(y, bins=bins)